In [ ]:
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt

In [ ]:
#employee_df = pd.read_csv(r'C:\Users\jakob\Desktop\Projektseminar\Data\employees.csv', delimiter= '|')
#absences_df = pd.read_csv(r'C:\Users\jakob\Desktop\Projektseminar\Data\absences.csv', delimiter='|')
#weather_df = pd.read_csv(r'C:/Users/jakob/Downloads/weather_wuerz.csv', delimiter=',')

employee_df = pd.read_csv("../../data/raw/employees.csv", delimiter= '|')
absences_df = pd.read_csv("../../data/raw/absences.csv", delimiter= '|')


In [ ]:
# drop NaN values and delete data for 2024 because to few data
absences_df = absences_df.dropna(subset=['start_date', 'end_date'])
absences_df = absences_df[absences_df['start_date'].str.contains('2024') == False]


Do following only once and load dataframe 'absences_each_missday_a_row' for faster access
Add rows for every booked absence, so each day is one person missing on one day.

In [ ]:
# Step 1: Create an empty list to store the rows of the new DataFrame
new_rows = []

# Step 2: Iterate over the original DataFrame row by row
for index, row in absences_df.iterrows():
    # Step 3: Generate a list of dates between 'start_date' and 'end_date' (inclusive)
    date_range = pd.date_range(start=row['start_date'], end=row['end_date'])
    
    # Step 4: Duplicate the row for each date in the list and add a new column with the date value
    for date in date_range:
        new_row = row[['employee_id', 'reason', 'allocation_id']].copy()
        new_row['date'] = date
        new_rows.append(new_row)

# Step 6: Create a new DataFrame using the list of duplicated rows
new_df = pd.DataFrame(new_rows)

# Reset the index of the new DataFrame
new_df.reset_index(drop=True, inplace=True)

# save df for faster reload/access
new_df.to_csv("../../data/interim/absences_each_missday_a_row.csv", index=False)

# Display the new DataFrame
new_df


In [ ]:
# read dataframe for faster access
my_df = pd.read_csv("../../data/interim/absences_each_missday_a_row.csv", delimiter=',')


In [ ]:
# order new_df by date
my_df = my_df.sort_values(by=['date'])
min_date = absences_df['start_date'].min()
max_date = absences_df['start_date'].max()

# Create a DataFrame with the date_range
date_range = pd.date_range(start=min_date, end=max_date)
all_dates = pd.DataFrame({'Date': date_range})
my_df['date'] = pd.to_datetime(my_df['date'])


In [ ]:
# join the two DataFrames
new_df = pd.merge(all_dates, my_df, how='left', left_on='Date', right_on='date')
#drop Date and employee_id
new_df = new_df.drop(['date', 'employee_id'], axis=1)
new_df

In [ ]:
aggregated_df = new_df.groupby('date').size().reset_index(name='count')

# Set 'date' column as the index
aggregated_df.set_index('date', inplace=True)

# Generate the complete date range from the minimum to the maximum date in the original dataframe
date_range = pd.date_range(start=new_df['date'].min(), end=new_df['date'].max(), freq='D')

# Reindex the aggregated dataframe to include missing dates and fill the NaN with zero
aggregated_df = aggregated_df.reindex(date_range, fill_value=0).reset_index()

aggregated_df

In [ ]:
aggregated_df['day_of_week'] = aggregated_df['index'].dt.dayofweek
aggregated_df['month'] = aggregated_df['index'].dt.month
aggregated_df['season'] = aggregated_df['index'].dt.quarter
# save dataframe for faster access
my_df = pd.to_csv("../../data/interim/absences_each_missday_a_row.csv", delimiter=',')
aggregated_df